In [46]:
import networkx as nx
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import Model, GRB

|Estudiante|Código|Correo|
|-----|-----|----|
|Juana Mejía Botero|20221512|j.mejia17|
|Daniela Ricaurte Echeverry|201822966|d.ricaurte|

# Proyecto - Entrega 3

# Table de Contenido
[[_TOC_]]

## Modelo Matemático

### Variables del Problema
- $y_{ij}$ variable binaria que representa el *estado de la llave de interconexión* entre i y j, donde esta será 1 si la llave está abierta y 0 si se encuentra cerrada.  
- $P_{ij}$ variable continua que representa el flujo de *potencia activa* entre los nodos $i$ y $j$.
- $Q_{ij}$ variable continua que representa el flujo de *potencia reactiva* entre los nodos $i$ y $j$.
- $I_{ij}$ variable continua que representa el flujo de corriente entre los nodos $i$ y $j$.
- $V_{i}$ variable continua  representa el voltaje en el nodo $i$.


### Parámetros del Problema

- $P_{i}^{D}$ representa la demanda de potencia activa en el nodo $i$ 

- $Q_{i}^{D}$  representa la demanda de potencia  reactiva en el nodo $i$.

- $R_{ij}$ representa la resistencia de la linea entre los nodos $i$ y $j$ 

- $X_{ij}$ representa la reactancia de la linea entre los nodos $i$ y $j$.

In [4]:
# Base de datos
file = open("C:\\Users\\danir\\OneDrive - Universidad de los andes\\Flujo de Redes\\Proyecto\\datos14.txt", "r")

print(file.read())
print()



read function: 
%% Datos del Sistema de 14 barras

% Datos globales
nref  = 14;         % nodo de referencia
vref  = 1.0;        % tensión en la subestación (pu)
vbase = 23.0;       % Tensión base (kV)
sbase = 100000;     % Potencia base (kVA)
tol   = 10^-8;      % Tolerancia del error permitido
vmin  = 0.93;       % Tensión mínima (pu)
vmax  = 1.05;       % Tensión máxima (pu)

% Base de impedancia
zbase = 100;

% Datos de ramas
%             de    para       R(%)     X(%)
ramos = [     14      13       7.50    10.00
              13      12       8.00    11.00
              13      11       9.00    18.00
              11      10       4.00     4.00
              14       9      11.00    11.00
               9       8       8.00    11.00
               9       7      11.00    11.00
               8       6      11.00    11.00
               8       5       8.00    11.00
              14       4      11.00    11.00
               4       3       9.00    12.00
               4       2  

In [40]:
import re

# Define a dictionary to store the values
values = {}
branches = []
bus_demand = []


# Define regular expressions to match the variables and their values
patterns = {
    'nref': r'nref\s*=\s*([\d.]+);',
    'vref': r'vref\s*=\s*([\d.]+);',
    'vbase': r'vbase\s*=\s*([\d.]+);',
    'sbase': r'sbase\s*=\s*([\d.]+);',
    'tol': r'tol\s*=\s*([\d.eE+-]+);',  # Updated for exponent notation
    'vmin': r'vmin\s*=\s*([\d.]+);',
    'vmax': r'vmax\s*=\s*([\d.]+);',
    'zbase': r'zbase\s*=\s*([\d.]+);',
}

branch_pattern = r'ramos\s*=\s*\[(.*?)\];'
bus_demand_pattern = r'barras\s*=\s*\[(.*?)\];'

# Read the content of the file
file_path = "C:\\Users\\danir\\OneDrive - Universidad de los andes\\Flujo de Redes\\Proyecto\\datos14.txt"
with open(file_path, "r") as file:
    text = file.read()

# Extract the values using regular expressions
for key, pattern in patterns.items():
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        if key == 'tol':
            print(key)
            base, exponent = match.group(1).split('e')
            values[key] = float(base) * 10 ** int(exponent)
        else:
            values[key] = float(match.group(1))
print (values)
# Print the extracted values
for key, value in values.items():
    print("items")
    print(f"{key}: {value}")
    

# Extract branch data using the pattern
match = re.search(branch_pattern, text, re.DOTALL)
if match:
    branch_data = match.group(1)

    # Split the data into individual lines and remove comments
    branch_lines = branch_data.split('\n')
    branch_lines = [line.strip() for line in branch_lines if not line.strip().startswith('%')]

    # Split each line into individual values and convert them to floats
    for line in branch_lines:
        valores = line.split()
        if len(valores) == 4:
            branch = [float(valor) for valor in valores]
            branches.append(branch)

# Print the extracted branch data
for branch in branches:
    print(branch)

match = re.search(bus_demand_pattern, text, re.DOTALL)
if match:
    bus_demand_data = match.group(1)

    # Split the data into individual lines and remove comments
    bus_demand_lines = bus_demand_data.split('\n')
    bus_demand_lines = [line.strip() for line in bus_demand_lines if not line.strip().startswith('%')]

    # Split each line into individual values and convert them to floats
    for line in bus_demand_lines:
        valores = line.split()
        if len(valores) >= 4:
            bus = [int(valores[0]), float(valores[1]), float(valores[2]), float(valores[3])]
            bus_demand.append(bus)

# Print the extracted bus demand data
for bus in bus_demand:
    print(bus)



{'nref': 14.0, 'vref': 1.0, 'vbase': 23.0, 'sbase': 100000.0, 'vmin': 0.93, 'vmax': 1.05, 'zbase': 100.0}
items
nref: 14.0
items
vref: 1.0
items
vbase: 23.0
items
sbase: 100000.0
items
vmin: 0.93
items
vmax: 1.05
items
zbase: 100.0
[14.0, 13.0, 7.5, 10.0]
[13.0, 12.0, 8.0, 11.0]
[13.0, 11.0, 9.0, 18.0]
[11.0, 10.0, 4.0, 4.0]
[14.0, 9.0, 11.0, 11.0]
[9.0, 8.0, 8.0, 11.0]
[9.0, 7.0, 11.0, 11.0]
[8.0, 6.0, 11.0, 11.0]
[8.0, 5.0, 8.0, 11.0]
[14.0, 4.0, 11.0, 11.0]
[4.0, 3.0, 9.0, 12.0]
[4.0, 2.0, 8.0, 11.0]
[2.0, 1.0, 4.0, 4.0]
[12.0, 6.0, 4.0, 4.0]
[7.0, 3.0, 4.0, 4.0]
[10.0, 1.0, 9.0, 12.0]
[14, 0.0, 0.0, 0.0]
[13, 2000.0, 1600.0, 0.0]
[12, 3000.0, 1500.0, 1100.0]
[11, 2000.0, 800.0, 1200.0]
[10, 1500.0, 1200.0, 0.0]
[9, 4000.0, 2700.0, 0.0]
[8, 5000.0, 3000.0, 1200.0]
[7, 1000.0, 900.0, 0.0]
[6, 600.0, 100.0, 600.0]
[5, 4500.0, 2000.0, 3700.0]
[4, 1000.0, 900.0, 0.0]
[3, 1000.0, 700.0, 1800.0]
[2, 1000.0, 900.0, 0.0]
[1, 2100.0, 1000.0, 1800.0]


In [44]:

nref=values.get("nref")
nodes = list(range(1,int(nref)+1))
print(nodes)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [47]:
# Crear un grafo dirigido
G = nx.DiGraph()

# Añadir nodos
for node in nodes:
    G.add_node(node)

### Función Objetivo

$$
Min \sum\limits_{ij \in {B}}\sum\limits_{t \in {T}} {{R_{ij}}I_{ij,t}^{sqr}};
$$


### Restiricciones del Problema

In [ ]:



# Datos
nodes = ['E1', 'E2', 'E3', 'N1', 'N2', 'N3', 'N4', 'Z1', 'Z2', 'Z3']
edges = [('E1', 'N1'), ('E2', 'N1'), ('E2', 'N2'), ('E3', 'N3'), ('N1', 'N3'), ('N1', 'N4'), 
         ('N2', 'N4'), ('N3', 'Z1'), ('N3', 'Z2'), ('N4', 'Z2'), ('N4', 'Z3')]
capacities = dict(zip(edges, [4000, 3000, 2000, 4500, 2500, 3500, 3000, 4000, 1500, 3000, 2500]))

# Crear el modelo
m = gp.Model("Evacuacion_Multiple")

# Variables
flow = m.addVars(edges, obj=-1, ub=capacities, name="flow")

# Restricciones de conservación de flujo
for node in nodes:
    # Nodos fuente
    if node in ['E1', 'E2', 'E3']:
        m.addConstr(flow.sum(node, '*') - flow.sum('*', node) == flow.sum(node, '*'), f"Fuente_{node}")
    # Nodos sumidero
    elif node in ['Z1', 'Z2', 'Z3']:
        m.addConstr(flow.sum('*', node) - flow.sum(node, '*') == flow.sum('*', node), f"Sumidero_{node}")
    # Otros nodos
    else:
        m.addConstr(flow.sum(node, '*') - flow.sum('*', node) == 0, f"Intermedio_{node}")

# Optimizar
m.optimize()
